In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator, train_discriminator, train_generator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [ ]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="0"
print(tf.test.is_gpu_available())

In [ ]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [ ]:
## reduce redundant features which can be assembled
dataset=df.drop(['NIHTotal','THD_ID','cortical_CT', 'subcortical_CT',
              'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
              'CT_left', 'CT_right',],axis=1)

In [ ]:
sc = MinMaxScaler()
dataset.loc[:,dataset.columns!='elapsed_class'] = sc.fit_transform(dataset.loc[:,dataset.columns!='elapsed_class'])


In [ ]:
## setting hyperparameter
latent_dim = dataset.shape[1]
epochs = 1000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
gen_opt = tf.keras.optimizers.Adam(0.0001,)
disc_opt = tf.keras.optimizers.Adam(0.0001,)

In [ ]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# separate to 0,1 dataset
data_1=dataset.loc[dataset['elapsed_class']==1]
data_0=dataset.loc[dataset['elapsed_class']==0]

In [ ]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0

    # resample the dataset
    data1_shape_0 = data_1.sample(data_0.shape[0])
    df_same_shape = pd.concat([data1_shape_0, data_0])

    # slices to data and labels
    df_same_ = df_same_shape.iloc[:, :-1].to_numpy()
    org_label = df_same_shape.iloc[:, -1].to_numpy().reshape(-1, 1)

    # create batch dataset
    training_dataset = tf.data.Dataset.from_tensor_slices((df_same_, org_label))\
        .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

    for data, labels in training_dataset:

        disc_loss += train_discriminator(data, generator,
                                         discriminator, disc_opt, latent_dim)

        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator(data, generator,
                                        discriminator, gen_opt, batch_size, latent_dim)
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1, time.time() - start,
                                                                               gen_loss / batch_size,
                                                                               disc_loss / (batch_size*n_critic)))

In [ ]:
noise = tf.random.normal([128*2, latent_dim])
zero_label=tf.zeros((128),)
one_label =tf.ones((128),)
all_labels=tf.reshape(tf.stack([zero_label,one_label]),(-1,1))

gen_=generator(noise,all_labels).numpy()

In [ ]:
output_dataset=pd.DataFrame(np.round(sc.inverse_transform(gen_)),columns=[
'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
       'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
       'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
       'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
       'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
       'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
       'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
       'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
       'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
       'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
       'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
       'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
       'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
       'NIHS_10_in', 'NIHS_11_in',
])
output_dataset

In [ ]:
# output_dataset.to_csv("../dataset/output_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')